# Tipos de dados no Spark: Simples e Complexo

### Estrutura Simples

| Tipo do Dado  | Equivalência em Python  | Instância no Spark                           |
| ------------- | ----------------------- | -------------------------------------------- |
| ByteType    | int               | ByteType()                                 |
| ShortType | int       | ShortType()                              |
| IntegerType      | int          | IntegerType()                                   |
| LongType     | int   | LongType()              |
| FloatType       | float                    | FloatType() |
| StringType    | str         | StringType()                         |
| BooleanType   | Bool | BooleanType() |
| DecimalType   | decimal.Decimal | DecimalType() |

### Estrutura Complexa

| Tipo do Dado  | Equivalência em Python  | Instância no Spark                           |
| ------------- | ----------------------- | -------------------------------------------- |
| BinaryType    | bytearray               | BinaryType()                                 |
| TimestampType | datetime.datetime       | TimestampType()                              |
| DateType      | datetime.date           | DateType()                                   |
| ArrayType     | List, tuple, or array   | ArrayType(dataType, [nullable])              |
| MapType       | dict                    | MapType(keyType, valueType, [nul<br/>lable]) |
| StructType    | List or tuple           | StructType([fields])                         |
| StructField   | Um objeto do tipo campo | StructField(name, dataType, [nul<br/>lable]) |

## Importando tipos

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType

## Exemplo de estrutura complexa

In [0]:
data = [
    (
        'Guilherme', [
            {
                'Telefones':
                {
                    'Fixo': '2137706213',
                    'Celular': '2196857513'
                }
            }
        ]
    ),
    (
        'Jéssica', [
            {
                'Telefones':
                {
                    'Fixo': '1122565427',
                    'Celular': '11984529368'
                }
            }
        ]
    )
]

## Schema Spark

> (Nome, [{Telefones: {Fixo: numero, Celular: numero}}])  
> tuple(string, list[dict[string: dict[string, string]]])

In [0]:
SCHEMA_TELEFONES = StructType(
    [
        StructField('Nome', StringType()),
        StructField('Contato',ArrayType(MapType(StringType(), StringType())))
    ]
)

## Dataframe Spark

In [0]:
df = spark.createDataFrame(data, SCHEMA_TELEFONES)

In [0]:
display(df)

Nome,Contato
Guilherme,"List(Map(Telefones -> {Celular=2196857513, Fixo=2137706213}))"
Jéssica,"List(Map(Telefones -> {Celular=11984529368, Fixo=1122565427}))"


## O Pandas aceita a mesma estrutura?

In [0]:
df.toPandas()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:145: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
 Not implemented type for Arrow list to pandas: map<string, string>
 warnings.warn(msg)

--------------------------------------------------------------------------- 
 ArrowNotImplementedError Traceback (most recent call last)
 <command-1683995409764404> in <module> 
 ----> 1 df . toPandas ( ) 

 /databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper (self, *args, **kwargs) 
 40 try : 
 41 start_time = time . time ( ) 
 ---> 42 return_val = func ( self , * args , ** kwargs ) 
 43 except Exception as e : 
 44 duration = ( time . time ( ) - start_time ) * 1000 

 /databricks/spark/python/pyspark/sql/pandas/conversion.py in toPandas (self) 
 120 # values, but we should use datetime.date to match the behavior with when 
 121 # Arrow optimization is disabled. 
 --> 122 pdf = table . to_pandas ( date_as_object = True ) 
 123 # Rename back to the original column names. 
 124 pdf . columns = self . columns

 /databricks/python/lib/python3.8/site-packages/pyarrow/array.pxi in pyarrow.lib._PandasConvertible.to_pandas () 

 /databricks/python/lib/python3.8/site-packages/pyarrow/table.pxi in pyarrow.lib.Table._to_pandas () 

 /databricks/python/lib/python3.8/site-packages/pyarrow/pandas_compat.py in table_to_blockmanager (options, table, categories, ignore_metadata, types_mapper) 
 787 _check_data_column_metadata_consistency ( all_columns ) 
 788 columns = _deserialize_column_index ( table , all_columns , column_indexes ) 
 --> 789 blocks = _table_to_blocks ( options , table , categories , ext_columns_dtypes ) 
 790 
 791 axes = [ columns , index ] 

 /databricks/python/lib/python3.8/site-packages/pyarrow/pandas_compat.py in _table_to_blocks (options, block_table, categories, extension_columns) 
 1126 # Convert an arrow table to Block from the internal pandas API 
 1127 columns = block_table . column_names
 -> 1128 result = pa.lib.table_to_blocks(options, block_table, categories,
 1129 list(extension_columns.keys()))
 1130 return [_reconstruct_block(item, columns, extension_columns)

 /databricks/python/lib/python3.8/site-packages/pyarrow/table.pxi in pyarrow.lib.table_to_blocks () 

 /databricks/python/lib/python3.8/site-packages/pyarrow/error.pxi in pyarrow.lib.check_status () 

 ArrowNotImplementedError : Not implemented type for Arrow list to pandas: map<string, string>

## Convertendo para estrutura simples do Pandas

In [0]:
from pyspark.sql.functions import to_json

In [0]:
pd = df.select(df.Nome,to_json(df.Contato).alias('Contato'))

In [0]:
pandinha = pd.toPandas()
display(pandinha)

Nome,Contato
Guilherme,"[{""Telefones"":""{Celular=2196857513, Fixo=2137706213}""}]"
Jéssica,"[{""Telefones"":""{Celular=11984529368, Fixo=1122565427}""}]"


## Convertendo para estrutura original dos dados

In [0]:
pandinha_dict = pandinha.to_dict('records')
pandinha_dict

Out[12]: [{'Nome': 'Guilherme',
 'Contato': '[{"Telefones":"{Celular=2196857513, Fixo=2137706213}"}]'},
 {'Nome': 'Jéssica',
 'Contato': '[{"Telefones":"{Celular=11984529368, Fixo=1122565427}"}]'}]

##### Toda a estrutura em 'Contatos' está como string é preciso converter para uma estrutura complexa e aninhada.

In [0]:
def converte_estrutura(texto: str):
    index_celular = texto.find('Celular') + 8
    index_fixo = texto.find('Fixo') + 5
    celular = texto[index_celular: index_celular + 10]
    fixo = texto[index_fixo: index_fixo + 10]
    
    return {'Celular': celular, 'Fixo': fixo}

In [0]:
original = [(pessoa['Nome'], [{'Telefones': converte_estrutura(pessoa['Contato'])}]) for pessoa in pandinha_dict]

for pessoas in original:
    print(pessoas)

('Guilherme', [{'Telefones': {'Celular': '2196857513', 'Fixo': '2137706213'}}])
('Jéssica', [{'Telefones': {'Celular': '1198452936', 'Fixo': '1122565427'}}])

## Recriando o dataframe com o Spark

In [0]:
df2 = spark.createDataFrame(original, SCHEMA_TELEFONES)

In [0]:
display(df2)

Nome,Contato
Guilherme,"List(Map(Telefones -> {Celular=2196857513, Fixo=2137706213}))"
Jéssica,"List(Map(Telefones -> {Celular=1198452936, Fixo=1122565427}))"


## Conclusão

O pandas não suporta estruturas complexas de dados, como solução ele converte toda a estrutura como uma string, deixando com que o desenvolvedor converta esses dados. Em contrapartida, o Spark mantém a estrutura dos dados oferecendo uma lista de tipagem mais ampla e estruturada.